### ✨ Google Maps traffic

Google maps es una herramienta que nos permite obtener el tráfico de una ubicación en tiempo real.

**🚀 Objetivo**

Obtener el tráfico de una zona específica utilizando únicamente la web de Google Maps.

**📚 Requisitos**

- Python 3.12
- Selenium


In [21]:
%pip install selenium pillow numpy opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fa/80/eb88edc2e2b11cd2dd2e56f1c80b5784d11d6e6b7f04a1145df64df40065/opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# needs latitude and longitude as parameters
MAPS_UTL = "https://www.google.com.mx/maps/@{},{},21z/data=!5m1!1e1?entry=ttu"

# example
# https://www.google.com.mx/maps/@20.6852493,-103.4423015,21z/data=!5m1!1e1?entry=ttu

# params explanation
# - latitude: 20.6852493. Represents the latitude of the location
# - longitude: -103.4423015. Represents the longitude of the location
# - 21z: Represents the zoom level of the map
# - data=!5m1!1e1?entry=ttu: Represents the data of the map
# - entry=ttu: Represents the entry point of the map

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image
import os
import numpy as np
import io
import cv2

In [6]:
def create_navigator_driver():
    chrome_options = Options()

    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_experimental_option(
        "excludeSwitches",
        ["ignore-certificate-errors", "disable-logging", "enable-logging"],
    )

    driver = webdriver.Chrome(
        options=chrome_options,
    )
    return driver

In [77]:
def get_maps_image_array(latitude, longitude, driver):
    url = MAPS_UTL.format(latitude, longitude)
    try:
        print("⬇️ Extracting image from", url)
        driver.get(url)

        screenshot_bytes = driver.get_screenshot_as_png()

        # transform bytes to image
        image = Image.open(io.BytesIO(screenshot_bytes))

        # transform image to numpy array
        image = np.array(image)

        return image
    except Exception as e:
        print("Error:", str(e))


def save_image_bytes_in_disk(image, folder, filename):
    target_path = os.path.join(folder, f"{filename}.png")
    print(f"⬇️ Saving image to {target_path}")

    image = Image.fromarray(image)
    os.makedirs(folder, exist_ok=True)
    image.save(target_path)

    print(f"✅ Image saved to {target_path}")

In [75]:
# test maps image

driver = create_navigator_driver()
image = get_maps_image_array(20.68522, -103.4425874, driver)
print("✨ image", image)

save_image_bytes_in_disk(image, "./", "traffic")

✨ image [[[170 185 201]
  [170 185 201]
  [170 185 201]
  ...
  [170 185 201]
  [170 185 201]
  [170 185 201]]

 [[170 185 201]
  [170 185 201]
  [170 185 201]
  ...
  [170 185 201]
  [170 185 201]
  [170 185 201]]

 [[170 185 201]
  [170 185 201]
  [170 185 201]
  ...
  [170 185 201]
  [170 185 201]
  [170 185 201]]

 ...

 [[248 247 247]
  [248 247 247]
  [248 247 247]
  ...
  [217 221 229]
  [220 224 231]
  [220 224 231]]

 [[248 247 247]
  [248 247 247]
  [248 247 247]
  ...
  [220 224 231]
  [220 224 231]
  [220 224 231]]

 [[226 229 233]
  [248 247 247]
  [248 247 247]
  ...
  [220 224 231]
  [220 224 231]
  [218 222 230]]]
⬇️ Saving image to ./traffic.png
✅ Image saved to ./traffic.png


In [ ]:
def crop_image_center(*, image: np.ndarray, center_percent=0.2):
    width, height = image.shape[:2]

    crop_width = int(width * center_percent)
    crop_height = int(height * center_percent)

    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2

    cropped_image = image[
        start_x : start_x + crop_width, start_y : start_y + crop_height
    ]

    return cropped_image


def extract_image_colors(*, image, from_rgb=False):
    """
    Return a np array. Keep this order: [green orange red red_wine]
    """
    # color ranges
    _LOWER_RED = np.array([0, 50, 200])
    _UPPER_RED = np.array([10, 255, 255])

    _LOWER_RED_WINE = np.array([0, 90, 90])
    _UPPER_RED_WINE = np.array([0, 230, 230])

    _LOWER_ORANGE = np.array([5, 50, 50])
    _UPPER_ORANGE = np.array([13, 250, 255])

    _LOWER_GREEN = np.array([40, 50, 50])
    _UPPER_GREEN = np.array([80, 255, 255])

    # convert to HSV if is rgb
    if from_rgb:
        _image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    else:
        _image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # mask
    mask_red = cv2.inRange(_image, _LOWER_RED, _UPPER_RED)
    mask_orange = cv2.inRange(_image, _LOWER_ORANGE, _UPPER_ORANGE)
    mask_green = cv2.inRange(_image, _LOWER_GREEN, _UPPER_GREEN)
    mask_red_wine = cv2.inRange(_image, _LOWER_RED_WINE, _UPPER_RED_WINE)

    red = cv2.countNonZero(mask_red) or 0
    orange = cv2.countNonZero(mask_orange) or 0
    green = cv2.countNonZero(mask_green) or 0
    red_wine = cv2.countNonZero(mask_red_wine) or 0

    return green, orange, red, red_wine


def remove_google_logo(image):
    x1, y1 = 365, 555
    x2, y2 = 435, 582
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 0), -1)

    return image


def get_image_color(original_image):
    green = 0
    orange = 0
    red = 0
    red_wine = 0
    image = None

    current_zoom = 0

    for zoom in [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
        current_zoom = zoom
        image = crop_image_center(image=original_image, center_percent=zoom)

        green, orange, red, red_wine = extract_image_colors(
            image=image,
            from_rgb=True,
        )

        total = green + orange + red + red_wine
        if total > 0:
            break

    return green, orange, red, red_wine, image, current_zoom

In [80]:
image = remove_google_logo(image)
save_image_bytes_in_disk(image, "./", "traffic_without_google_logo")

green, orange, red, red_wine, crop_image, zoom = get_image_color(image)
print(
    "✨ Colors:",
    {
        "green": green,
        "orange": orange,
        "red": red,
        "red_wine": red_wine,
        "zoom": zoom,
    },
)
save_image_bytes_in_disk(crop_image, "./", "crop_image")

⬇️ Saving image to ./traffic_without_google_logo.png
✅ Image saved to ./traffic_without_google_logo.png
✨ Colors: {'green': 0, 'orange': 0, 'red': 3744, 'red_wine': 50, 'zoom': 0.3}
⬇️ Saving image to ./crop_image.png
✅ Image saved to ./crop_image.png


In [79]:
def calculate_traffic_feature(
    green: float,
    orange: float,
    red: float,
    red_wine: float,
    green_weight: float = 0.125,
    orange_weight: float = 0.25,
    red_weight: float = 0.5,
    red_wine_weight: float = 1,
) -> int:
    """
    Calculates a weighted percentage. By default the weights are exponential.
    """
    score = (
        green * green_weight
        + orange * orange_weight
        + red * red_weight
        + red_wine * red_wine_weight
    )

    max_score = green + orange + red + red

    if max_score == 0:
        return 0.0

    normalized = score / max_score

    return score, max_score, normalized

In [78]:
score, max_score, feature = calculate_traffic_feature(
    green=green,
    orange=orange,
    red=red,
    red_wine=red_wine,
    green_weight=0.05,
    orange_weight=0.1,
    red_weight=0.25,
    red_wine_weight=0.6,
)
print(
    "✨ feature:",
    f"({feature * 100:.2f}%)",
    {
        "feature": feature,
        "score": score,
        "max_score": max_score,
    },
)

✨ feature: (12.90%) {'feature': 0.12900641025641027, 'score': 966.0, 'max_score': 7488}


In [ ]:
coords = [
    {
        "lat": 20.6668671,
        "lng": -103.4512208,
    },
    {
        "lat": 20.6309778,
        "lng": -103.3360105,
    },
]

driver = create_navigator_driver()
features = []

for coord in coords:
    print("⬇️ Extracting data from", coord)
    image = get_maps_image_array(coord["lat"], coord["lng"], driver)
    image = remove_google_logo(image)
    green, orange, red, red_wine, crop_image, zoom = get_image_color(image)

    score, max_score, feature = calculate_traffic_feature(
        green=green,
        orange=orange,
        red=red,
        red_wine=red_wine,
        green_weight=0.05,
        orange_weight=0.1,
        red_weight=0.25,
        red_wine_weight=0.6,
    )
    print(
        "✨ feature:",
        f"({feature * 100:.2f}%)",
        {
            "feature": feature,
            "score": score,
            "max_score": max_score,
        },
    )

    features.append(
        {
            "coord": coord,
            "feature": feature,
            "score": score,
            "max_score": max_score,
        }
    )

print(features)


⬇️ Extracting data from {'lat': 20.6668671, 'lng': -103.4512208}
⬇️ Extracting image from https://www.google.com.mx/maps/@20.6668671,-103.4512208,21z/data=!5m1!1e1?entry=ttu
✨ feature: (5.00%) {'feature': 0.05, 'score': 105.95, 'max_score': 2119}
⬇️ Extracting data from {'lat': 20.6309778, 'lng': -103.3360105}
⬇️ Extracting image from https://www.google.com.mx/maps/@20.6309778,-103.3360105,21z/data=!5m1!1e1?entry=ttu
✨ feature: (13.05%) {'feature': 0.1305235903337169, 'score': 680.55, 'max_score': 5214}
[{'coord': {'lat': 20.6668671, 'lng': -103.4512208}, 'feature': 0.05, 'score': 105.95, 'max_score': 2119}, {'coord': {'lat': 20.6309778, 'lng': -103.3360105}, 'feature': 0.1305235903337169, 'score': 680.55, 'max_score': 5214}]
